# Testing Recommender Functions

This notebook installs the required dependencies and tests the recommender functions defined in your modules (e.g. `utils.py` and `recommender.py`). It loads the models via the `get_models()` function and then tests various ensemble methods.

In [1]:
# Install required dependencies
#!pip install --upgrade pip
#!pip install numpy scikit-learn tensorflow keras fastapi torch transformers

# If your project has a requirements.txt file, you can also use:
# !pip install --no-cache-dir -r requirements.txt

In [ ]:
from cluster_utils import main
#def main(process_dfs = False, process_behaviors = False, data_dir = 'dataset/train/', valid_data_dir = 'dataset/valid/', zip_file = f"MINDlarge_train.zip", valid_zip_file = f"MINDlarge_dev.zip"):
#main()
main(True, True, 'dataset/small/train/', 'dataset/small/valid/', f"MINDsmall_train.zip", f"MINDsmall_dev.zip", 'small_user_category_profiles.pkl', 'small_user_cluster_df.pkl')

2025-03-26 13:43:47.557215: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-26 13:43:47.575677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742996627.589183    1762 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742996627.593566    1762 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-26 13:43:47.611184: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loaded news data:
   NewsID   Category      SubCategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               Title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            Abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             URL  \
0  https:

100%|██████████| 156965/156965 [00:14<00:00, 11131.92it/s]


Created train_df with 5843444 samples.
Columns in train_df:
Index(['UserID', 'HistoryTitles', 'CandidateTitleTokens', 'Label'], dtype='object')
All 'UserID's are present in user_category_profiles.
Assigned clusters to users. Number of clusters: 3
Assigning cluster labels to train_df using map...
All samples have cluster assignments.
Cluster 0: 5039111 test samples.
Cluster 1: 102103 test samples.
Cluster 2: 702230 test samples.

After label balancing (0 vs 1):
Label
0    14370
1    14370
Name: count, dtype: int64
Balanced cluster sizes:
Cluster 0: 8877 samples
Cluster 1: 10357 samples
Cluster 2: 9506 samples
Balanced dataset:
Cluster
1    10357
2     9506
0     8877
Name: count, dtype: int64
Original size: 28740
Sampled size: 28740
Columns in sampled train_df:
Index(['UserID', 'HistoryTitles', 'CandidateTitleTokens', 'Label', 'Cluster'], dtype='object')
Cluster:4343     1
8256     0
6526     1
25186    2
24283    2
        ..
21575    2
5390     1
860      0
15795    1
23654    0
Name:

100%|██████████| 156965/156965 [00:15<00:00, 10072.84it/s]


Created train_df with 5843444 samples.
Columns in train_df:
Index(['UserID', 'HistoryTitles', 'CandidateTitleTokens', 'Label'], dtype='object')
All 'UserID's are present in user_category_profiles.
Assigned clusters to users. Number of clusters: 3
Assigning cluster labels to train_df using map...
All samples have cluster assignments.
Cluster 0: 5039111 test samples.
Cluster 1: 102103 test samples.
Cluster 2: 702230 test samples.

After label balancing (0 vs 1):
Label
0    14370
1    14370
Name: count, dtype: int64
Balanced cluster sizes:
Cluster 0: 8877 samples
Cluster 1: 10357 samples
Cluster 2: 9506 samples
Balanced dataset:
Cluster
1    10357
2     9506
0     8877
Name: count, dtype: int64
Original size: 28740
Sampled size: 28740
Columns in sampled train_df:
Index(['UserID', 'HistoryTitles', 'CandidateTitleTokens', 'Label', 'Cluster'], dtype='object')
Cluster:4343     1
8256     0
6526     1
25186    2
24283    2
        ..
21575    2
5390     1
860      0
15795    1
23654    0
Name:

2025-03-26 13:45:26.697269: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'user_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 46 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Loading model for Cluster 1 from fastformer_cluster_1_full_balanced_1_epoch.keras
.cache
.ipynb_checkpoints
.Trash-0
backend copy 2.py
backend copy.py
backend-flask-unused.py
backend.py
cluster_profile-Copy1.log
cluster_profile.log
cluster_utils.py
data
dataset
detailed_log.log
Dockerfile
downloads
experiment_results.csv
fastapi copy.py
fastapi2.py
fastformer.json
fastformer_clusters.ipynb
fastformer_cluster_0_full_balanced_1_epoch.h5
fastformer_cluster_0_full_balanced_1_epoch.json
fastformer_cluster_0_full_balanced_1_epoch.keras
fastformer_cluster_0_full_balanced_1_epoch.weights.h5
fastformer_cluster_1_full_balanced_1_epoch.keras
fastformer_cluster_2_full_balanced_1_epoch.keras
fastformer_model.py
gdrive.py
models
models.py
my_log.py
recommender.py
requirements.txt
results
results.csv
small_behaviors_df_processed.pkl
small_user_category_profiles.pkl
stage1_candidates.csv
test_recommender.ipynb
test_recommender.py
tfidf_clicked_articles.csv
tfidf_similarity_distribution.png
tokenizer.

/app/cluster_utils.py:83: UserWarning: no explicit representation of timezones available for np.datetime64
  (behaviors_df['Time'] <= np.datetime64(cutoff_time))
/app/cluster_utils.py:83: UserWarning: no explicit representation of timezones available for np.datetime64
  (behaviors_df['Time'] <= np.datetime64(cutoff_time))
/app/cluster_utils.py:83: UserWarning: no explicit representation of timezones available for np.datetime64
  (behaviors_df['Time'] <= np.datetime64(cutoff_time))
/app/cluster_utils.py:83: UserWarning: no explicit representation of timezones available for np.datetime64
  (behaviors_df['Time'] <= np.datetime64(cutoff_time))
/app/cluster_utils.py:83: UserWarning: no explicit representation of timezones available for np.datetime64
  (behaviors_df['Time'] <= np.datetime64(cutoff_time))
/app/cluster_utils.py:83: UserWarning: no explicit representation of timezones available for np.datetime64
  (behaviors_df['Time'] <= np.datetime64(cutoff_time))
/app/cluster_utils.py:83: Us

In [ ]:
fail here!!

In [ ]:
from utils import get_models, run_experiments_modular
models_dict, news_df, behaviors_df, tokenizer = get_models()
    
# Fit a TF-IDF vectorizer on the news combined text.
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(news_df["CombinedText"])

results_df = run_experiments_modular(
    behaviors_df, news_df, models_dict, tokenizer,
    tfidf_vectorizer=tfidf_vectorizer,
    max_candidates=-1, test_user_count=1, n_dates=3,
    timeframe_hours=24*10, k=100, ensemble_method="bagging",
    min_tfidf_similarity=0.02
    )

In [ ]:



from utils import get_models, run_experiments
models_dict, news_df, behaviors_df, tokenizer = get_models()
user_data = behaviors_df[behaviors_df['UserID'] == "U379278"]
print(user_data[['Time', 'Impressions']])

run_experiments(behaviors_df, models_dict, -1, 1, 5, 24*10, 20, filter_method="both", min_tfidf_similarity=0.02)
#def run_experiments(behaviors_df, models_dict, max_candidates=-1, test_user_count=1, n_dates=3, timeframe_hours=1, k=5, filter_method="both", min_tfidf_similarity=0.02, tfidf_vectorizer=None):


In [ ]:
thresholds = np.linspace(0.0, 1.0, 21)
performance = []

for thr in thresholds:
    filtered_candidates = [score for score in candidate_similarities if score >= thr]
    # Here, compute your chosen metric (e.g., F1 or Precision@K) using the filtered candidates.
    metric_value = compute_metric(filtered_candidates, ground_truth_clicks)
    performance.append(metric_value)

plt.plot(thresholds, performance, marker='o')
plt.xlabel("TF‑IDF Similarity Threshold")
plt.ylabel("Metric (e.g., F1‑score)")
plt.title("Threshold Tuning for Candidate Filtering")
plt.show()

In [ ]:
# Import required modules
import numpy as np
from utils import get_models  # Ensure these are in your PYTHONPATH
from recommender import ensemble_bagging, ensemble_boosting, train_stacking_meta_model, ensemble_stacking, hybrid_ensemble, tokenize_input

# For demonstration, we assume get_models() returns a dictionary of models for clusters 0, 1, 2, etc.
print("Modules imported successfully.")

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sys
import tensorflow as tf

# Remap standalone keras modules to tensorflow.keras
sys.modules["keras.preprocessing.text"] = tf.keras.preprocessing.text
sys.modules["keras.preprocessing.sequence"] = tf.keras.preprocessing.sequence
sys.modules["keras.utils"] = tf.keras.utils

news_file = "news.tsv"
behaviors_file = "behaviors.tsv"
data_dir = 'dataset/train/'  # Adjust path as necessary
valid_data_dir = 'dataset/valid/'  # Adjust path as necessary
news_path = os.path.join(data_dir, news_file)
behaviors_path = os.path.join(data_dir, behaviors_file)

# Set maximum lengths (should match your model settings)
max_history_length = 50
max_title_length = 30

# Load the pre-saved tokenizer (assumes you already created and saved it)
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Load MIND test data (adjust file paths as necessary)
# Assume news.tsv contains columns: NewsID, Category, SubCategory, Title, Abstract, URL, TitleEntities, AbstractEntities
news_df = pd.read_csv(news_path, sep='\t', 
                      names=['NewsID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'TitleEntities', 'AbstractEntities'])
# Assume behaviors_test.tsv contains: ImpressionID, UserID, Time, HistoryText, Impressions
behaviors_df = pd.read_csv(behaviors_path, sep='\t', 
                           names=['ImpressionID', 'UserID', 'Time', 'HistoryText', 'Impressions'])

# Create a dictionary mapping NewsID to Title (or CombinedText if available)
news_dict = dict(zip(news_df['NewsID'], news_df['Title']))

# Select one sample from the test behaviors
sample = behaviors_df.iloc[0]

# Process history: split the HistoryText (a space-separated string of NewsIDs)
history_text = sample['HistoryText']
history_ids = history_text.split() if pd.notna(history_text) else []

# Retrieve the title for each news ID in the history (default to empty string if missing)
history_titles = [news_dict.get(nid, "") for nid in history_ids]

# Convert history titles to sequences using the tokenizer
history_sequences = tokenizer.texts_to_sequences(history_titles)
# Pad each sequence to max_title_length
history_padded = pad_sequences(history_sequences, maxlen=max_title_length, 
                               padding='post', truncating='post', value=0)

# Ensure the history has exactly max_history_length rows:
if history_padded.shape[0] < max_history_length:
    # Pre-pad with zeros if there are fewer history items
    pad_rows = np.zeros((max_history_length - history_padded.shape[0], max_title_length), dtype=int)
    history_padded = np.vstack([pad_rows, history_padded])
else:
    # If too many, take the last max_history_length items
    history_padded = history_padded[-max_history_length:]

# Process candidate: the "Impressions" column is a space-separated list like "newsID-label newsID-label ..."
impressions = sample['Impressions']
first_candidate = impressions.split()[0]  # take the first candidate
candidate_news_id = first_candidate.split('-')[0]
candidate_title = news_dict.get(candidate_news_id, "")
candidate_sequence = tokenizer.texts_to_sequences([candidate_title])
candidate_padded = pad_sequences(candidate_sequence, maxlen=max_title_length, 
                                 padding='post', truncating='post', value=0)[0]

# Convert to TensorFlow tensors
history_tensor = tf.convert_to_tensor([history_padded], dtype=tf.int32)  # shape: (1, max_history_length, max_title_length)
candidate_tensor = tf.convert_to_tensor([candidate_padded], dtype=tf.int32)  # shape: (1, max_title_length)

print("History tensor shape:", history_tensor.shape)
print("Candidate tensor shape:", candidate_tensor.shape)

# Load ensemble models using the get_models function
print("Loading models...")
models_dict = get_models()
print("Models loaded:", models_dict.keys())

# Test ensemble bagging
bagging_pred = ensemble_bagging(history_tensor, candidate_tensor, models_dict)
print("Ensemble Bagging Prediction:", bagging_pred)

# Test ensemble boosting with dummy error values
dummy_errors = np.array([0.2, 0.15, 0.25])
boosting_pred = ensemble_boosting(history_tensor, candidate_tensor, models_dict, dummy_errors)
print("Ensemble Boosting Prediction:", boosting_pred)

# Test ensemble stacking with dummy training data
X_train_dummy = np.array([
    [0.80, 0.75, 0.85],
    [0.55, 0.60, 0.50],
    [0.30, 0.35, 0.25],
    [0.20, 0.25, 0.15]
])
y_train_dummy = np.array([1, 0, 1, 0])
meta_model = train_stacking_meta_model(X_train_dummy, y_train_dummy)
stacking_pred = ensemble_stacking(history_tensor, candidate_tensor, models_dict, meta_model)
print("Ensemble Stacking Prediction:", stacking_pred)

# Test hybrid ensemble
hybrid_pred = hybrid_ensemble(history_tensor, candidate_tensor, models_dict, dummy_errors, meta_model)
print("Hybrid Ensemble Prediction:", hybrid_pred)

### Next Steps

You can now develop and test your recommendation functions independently of the FastAPI backend. 

For further debugging, you might want to add additional print statements or assertions within your recommender functions.